In [1]:
import pandas as pd
import os
import json
import ast
from utils import *

In [2]:
pairs = pd.read_csv("trait_condition_pairs.csv")

In [3]:
all_traits = pd.read_csv("all_traits.csv")["Trait"].tolist()
all_traits = [normalize_trait(at) for at in all_traits]

In [4]:
rel = pd.read_csv("trait_related_genes.csv")
rel['Related_Genes'] = rel['Related_Genes'].apply(ast.literal_eval)
t2g = pd.Series(rel['Related_Genes'].values, index=rel['Trait']).to_dict()  # the mapping from trait to genes

In [ ]:
gene_info_path = './trait_related_genes.csv'
data_root = '/home/techt/Desktop/a4s/gold_subset'
output_root = './output'

suc = 0
condition=None
performance_ls = []

for trait in all_traits:
    print(f"Trait {trait} only")
    output_dir = os.path.join(output_root, trait)
    os.makedirs(output_dir, exist_ok=True)
    try:
        trait_data, _, _ = select_and_load_cohort(data_root, trait, is_two_step=False)
        redundant_col = ['Age', 'Gender']
        trait_data = trait_data.drop(columns=[redundant_col], errors="ignore")

        Y = trait_data[trait].values
        Z = None
        X = trait_data.drop(columns=[trait]).values

        has_batch_effect = detect_batch_effect(X)
        if False: #has_batch_effect:
            model_constructor = VariableSelection
            fixed_params = {'modified': True}
            tune_params = {'lamda': [3e-6, 1e-5, 3e-5, 1e-4, 3e-4, 1e-3]}
        else:
            model_constructor = Lasso
            fixed_params = {'random_state': 42}
            tune_params = {'alpha': [0.03, 0.1, 0.3, 1, 3, 10]}

        best_config, best_performance_metrics = tune_hyperparameters(model_constructor, fixed_params, tune_params, X, Y, trait_data.columns, trait, gene_info_path, condition, Z)
        performance_ls.append(best_performance_metrics)
        model = ResidualizationRegressor(model_constructor, best_config)
        normalized_X, _ = normalize_data(X)
        model.fit(normalized_X, Y)

        var_names = trait_data.columns.tolist()
        threshold = 0.05
        interpret_result(model, var_names, trait, condition, threshold=threshold, save_output=True,
                         output_dir=output_dir)
        suc += 1
    except:
        print(f"Error processing trait {trait}")
        continue


Trait Breast_Cancer only
The cross-validation performance: {'prediction': {'accuracy': 0.9934156378600824}, 'selection': {'precision': 0.369337758811443, 'precision_at_50': 0.369337758811443, 'recall': 0.003023001095290252, 'f1_score': 0.0059968828169693535, 'jaccard': 0.0030074858047642655, 'jaccard2': 0.0}}
The cross-validation performance: {'prediction': {'accuracy': 0.9275720164609054}, 'selection': {'precision': 0.5457575757575758, 'precision_at_50': 0.5457575757575758, 'recall': 0.001226725082146769, 'f1_score': 0.002447877392985183, 'jaccard': 0.0012254882944857672, 'jaccard2': 0.0}}
The cross-validation performance: {'prediction': {'accuracy': 0.9069958847736626}, 'selection': {'precision': 0.0, 'precision_at_50': 0.0, 'recall': 0.0, 'f1_score': 0.0, 'jaccard': 0.0, 'jaccard2': 0.0}}
The cross-validation performance: {'prediction': {'accuracy': 0.9069958847736626}, 'selection': {'precision': 0.0, 'precision_at_50': 0.0, 'recall': 0.0, 'f1_score': 0.0, 'jaccard': 0.0, 'jaccard2'

/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.225e+00, tolerance: 3.776e-01
  model = cd_fast.enet_coordinate_descent(


The gene info file does not contain genes related to the feature 'Heart_rate'.
Error processing trait Heart_rate
Trait Height only
Error processing trait Height
Trait Li-Fraumeni_Syndrome only
Error processing trait Li-Fraumeni_Syndrome
Trait Lupus_(Systemic_Lupus_Erythematosus) only
Error processing trait Lupus_(Systemic_Lupus_Erythematosus)
Trait Multiple_Endocrine_Neoplasia_Type_2 only
Error processing trait Multiple_Endocrine_Neoplasia_Type_2
Trait Obesity only
The cross-validation performance: {'prediction': {'accuracy': 0.9199999999999999}, 'selection': {'precision': 0.1598972922502334, 'precision_at_50': 0.1598972922502334, 'recall': 0.005303030303030303, 'f1_score': 0.010265479131345814, 'jaccard': 0.0051607858058114325, 'jaccard2': 0.011512553582363748}}
The cross-validation performance: {'prediction': {'accuracy': 0.8933333333333333}, 'selection': {'precision': 0.1627941176470588, 'precision_at_50': 0.1627941176470588, 'recall': 0.0026515151515151517, 'f1_score': 0.0052172888

/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.513e-02, tolerance: 2.778e-03
  model = cd_fast.enet_coordinate_descent(
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.640e-03, tolerance: 2.799e-03
  model = cd_fast.enet_coordinate_descent(
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing reg

The cross-validation performance: {'prediction': {'accuracy': 1.0}, 'selection': {'precision': 0.0, 'precision_at_50': 0.0, 'recall': 0.0, 'f1_score': 0.0, 'jaccard': 0.0, 'jaccard2': 0.0}}


...

The cross-validation performance: {'prediction': {'accuracy': 0.9928571428571429}, 'selection': {'precision': 0.0, 'precision_at_50': 0.0, 'recall': 0.0, 'f1_score': 0.0, 'jaccard': 0.0, 'jaccard2': 0.0}}
The cross-validation performance: {'prediction': {'accuracy': 0.9928571428571429}, 'selection': {'precision': 0.0, 'precision_at_50': 0.0, 'recall': 0.0, 'f1_score': 0.0, 'jaccard': 0.0, 'jaccard2': 0.0}}
The cross-validation performance: {'prediction': {'accuracy': 0.43571428571428567}, 'selection': {'precision': 0.0, 'precision_at_50': 0.0, 'recall': 0.0, 'f1_score': 0.0, 'jaccard': 0.0, 'jaccard2': 0.0}}
The cross-validation performance: {'prediction': {'accuracy': 0.43571428571428567}, 'selection': {'precision': 0.0, 'precision_at_50': 0.0, 'recall': 0.0, 'f1_score': 0.0, 'jaccard': 0.0, 'jaccard2': 0.0}}
The cross-validation performance: {'prediction': {'accuracy': 0.3928571428571429}, 'selection': {'precision': 0.0, 'precision_at_50': 0.0, 'recall': 0.0, 'f1_score': 0.0, 'jaccar

/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.268e-02, tolerance: 3.499e-03
  model = cd_fast.enet_coordinate_descent(


Trait Bladder_Cancer only
The cross-validation performance: {'prediction': {'accuracy': 0.9811764705882353}, 'selection': {'precision': 0.22175259989053092, 'precision_at_50': 0.22175259989053092, 'recall': 0.004041916167664671, 'f1_score': 0.007935400515268805, 'jaccard': 0.0039836368470745785, 'jaccard2': 0.0}}
The cross-validation performance: {'prediction': {'accuracy': 0.9552941176470588}, 'selection': {'precision': 0.24666666666666667, 'precision_at_50': 0.24666666666666667, 'recall': 0.0005988023952095808, 'f1_score': 0.0011942537820866389, 'jaccard': 0.0005977958581729793, 'jaccard2': 0.0}}
The cross-validation performance: {'prediction': {'accuracy': 0.9552941176470588}, 'selection': {'precision': 0.0, 'precision_at_50': 0.0, 'recall': 0.0, 'f1_score': 0.0, 'jaccard': 0.0, 'jaccard2': 0.0}}
The cross-validation performance: {'prediction': {'accuracy': 0.9552941176470588}, 'selection': {'precision': 0.0, 'precision_at_50': 0.0, 'recall': 0.0, 'f1_score': 0.0, 'jaccard': 0.0, 'j

/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.851e-03, tolerance: 1.496e-03
  model = cd_fast.enet_coordinate_descent(
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.701e-03, tolerance: 1.575e-03
  model = cd_fast.enet_coordinate_descent(


The cross-validation performance: {'prediction': {'accuracy': 1.0}, 'selection': {'precision': 0.0, 'precision_at_50': 0.0, 'recall': 0.0, 'f1_score': 0.0, 'jaccard': 0.0, 'jaccard2': 0.0}}
The cross-validation performance: {'prediction': {'accuracy': 1.0}, 'selection': {'precision': 0.0, 'precision_at_50': 0.0, 'recall': 0.0, 'f1_score': 0.0, 'jaccard': 0.0, 'jaccard2': 0.0}}
The cross-validation performance: {'prediction': {'accuracy': 0.7777777777777778}, 'selection': {'precision': 0.0, 'precision_at_50': 0.0, 'recall': 0.0, 'f1_score': 0.0, 'jaccard': 0.0, 'jaccard2': 0.0}}
The cross-validation performance: {'prediction': {'accuracy': 0.7222222222222221}, 'selection': {'precision': 0.0, 'precision_at_50': 0.0, 'recall': 0.0, 'f1_score': 0.0, 'jaccard': 0.0, 'jaccard2': 0.0}}
The cross-validation performance: {'prediction': {'accuracy': 0.7333333333333334}, 'selection': {'precision': 0.0, 'precision_at_50': 0.0, 'recall': 0.0, 'f1_score': 0.0, 'jaccard': 0.0, 'jaccard2': 0.0}}
The c

In [ ]:
prec = [p['selection']['precision'] for p in performance_ls]

In [ ]:
np.mean(prec)

In [ ]:
performance_ls

In [ ]:
prec = [p['selection']['jaccard'] for p in performance_ls]
np.mean(prec)

In [ ]:
gene_info_path = './trait_related_genes.csv'
data_root = '/home/techt/Desktop/a4s/gold_subset'
output_root = './output'

for i, (index, row) in enumerate(pairs.iterrows()):
    try:
        trait, condition = row['Trait'], row['Condition']
        output_dir = os.path.join(output_root, trait)
        os.makedirs(output_dir, exist_ok=True)

        if condition in ['Age', 'Gender']:
            continue
            trait_data, _, _ = select_and_load_cohort(data_root, trait, is_two_step=False)
            redundant_col = 'Age' if condition == 'Gender' else 'Gender'
            if redundant_col in trait_data.columns:
                trait_data = trait_data.drop(columns=[redundant_col])

            Y = trait_data[trait].values
            Z = trait_data[condition].values
            X = trait_data.drop(columns=[trait, condition]).values

            has_batch_effect = detect_batch_effect(X)
            if has_batch_effect:
                model_constructor = VariableSelection
                fixed_params = {'modified': True}
                tune_params = {'lamda': [3e-6, 1e-5, 3e-5, 1e-4, 3e-4, 1e-3]}
            else:
                model_constructor = Lasso
                fixed_params = {'random_state': 42}
                tune_params = {'alpha': [1e-2, 1e-1, 1, 10]}

            best_config, best_performance_metrics = tune_hyperparameters(model_constructor, fixed_params, tune_params, X, Y, trait_data.columns, trait, gene_info_path, condition, Z)

            model = ResidualizationRegressor(model_constructor, best_config)
            normalized_X, _ = normalize_data(X)
            normalized_Z, _ = normalize_data(Z)
            model.fit(normalized_X, Y, normalized_Z)

            var_names = trait_data.columns.tolist()
            threshold = 0.05
            interpret_result(model, var_names, trait, condition, threshold=threshold, save_output=True,
                             output_dir=output_dir)

        else:
            # if trait != 'Inflammatory_Disorders' or condition != 'Lung_Cancer': continue
            trait_data, condition_data, regressors = select_and_load_cohort(data_root, trait, condition, is_two_step=True, gene_info_path=gene_info_path)
            if regressors is None:
                print(f'No gene regressors for trait {trait} and condition {condition}')
                continue

            print("Common gene regressors for condition and trait", regressors)
            X_condition = condition_data[regressors].values
            Y_condition = condition_data[condition].values

            condition_type = 'binary' if len(np.unique(Y_condition)) == 2 else 'continuous'

            if condition_type == 'binary':
                if X_condition.shape[1] > X_condition.shape[0]:
                    model = LogisticRegression(penalty='l1', solver='liblinear', random_state=42)
                else:
                    model = LogisticRegression()
            else:
                if X_condition.shape[1] > X_condition.shape[0]:
                    model = Lasso()
                else:
                    model = LinearRegression()

            normalized_X_condition, _ = normalize_data(X_condition)
            model.fit(normalized_X_condition, Y_condition)

            regressors_in_trait = trait_data[regressors].values
            normalized_regressors_in_trait, _ = normalize_data(regressors_in_trait)
            if condition_type == 'binary':
                predicted_condition = model.predict_proba(normalized_regressors_in_trait)[:, 1]
            else:
                predicted_condition = model.predict(normalized_regressors_in_trait)

            trait_data[condition] = predicted_condition
            trait_data = trait_data.drop(columns=regressors)
            trait_data = trait_data.drop(columns=['Age', 'Gender'], errors='ignore')

            Y = trait_data[trait].values
            Z = trait_data[condition].values
            X = trait_data.drop(columns=[trait, condition]).values

            has_batch_effect = detect_batch_effect(X)
            if has_batch_effect:
                model_constructor = VariableSelection
                fixed_params = {'modified': True}
                tune_params = {'lamda': [3e-6, 1e-5, 3e-5, 1e-4, 3e-4, 1e-3]}
            else:
                model_constructor = Lasso
                fixed_params = {'random_state': 42}
                tune_params = {'alpha': [1e-2, 1e-1, 1, 10]}

            best_config, best_performance_metrics = tune_hyperparameters(model_constructor, fixed_params, tune_params, X, Y, trait_data.columns, trait, gene_info_path, condition, Z)

            model = ResidualizationRegressor(model_constructor, best_config)
            normalized_X, _ = normalize_data(X)
            normalized_Z, _ = normalize_data(Z)
            model.fit(normalized_X, Y, normalized_Z)

            var_names = trait_data.columns.tolist()
            threshold = 0.05
            interpret_result(model, var_names, trait, condition, threshold=threshold, save_output=True,
                             output_dir=output_dir)
    except Exception as e:
        print(f"Error processing row {i}, for the trait '{trait}' and the condition '{condition}'\n: {e}")
        continue
